In [4]:
import itertools
import os
import random
from abc import ABCMeta, abstractmethod
from contextlib import redirect_stdout
from sys import stderr
import matplotlib.pyplot as plt
import numpy as np
import math

from pycocotools.coco import COCO
from skimage.io import imread

# DATA_DIR = "/mnt/ssd1/processed/industry-data/project-max/ml/cloud-factory-data/with-bg/multi-label-datasets/forwards/notebook-ready"

DATA_DIR = "/mnt/ssd1/processed/industry-data/project-max/ml/cloud-factory-data/with-bg/multi-label-datasets/forwards/earl-originals/"

# Coco Category Translator
For using earl outputs directly

In [5]:
class CocoCategoryTranslator(AnnotationTranslator):
        def filter(self, annotation):
            '''Filter out non-caption annotations'''
            return True

        def translate(self, annotation):
            '''Return a list of strings'''
            # for img_id in coco.getImgIds():
            #     for ann in coco.getAnnIds(image_ids = [img_id])

            return [annotation['category_id']]

# Coco data format

In [2]:
from abyss_deep_learning.datasets.coco import ImageClassificationDataset
from abyss_deep_learning.datasets.translators import AnnotationTranslator
from abyss_deep_learning.datasets.translators import CocoCaptionTranslator, AbyssCaptionTranslator

Using TensorFlow backend.


# Test COCO Realisations

## Data type: Classification, Task: Classification

In [3]:
# train_ds = ImageClassificationDataset(
#     os.path.join(DATA_DIR, "train-nb.json"),
#     image_dir=DATA_DIR,
#     cached=False)

# val_ds = ImageClassificationDataset(
#     os.path.join(DATA_DIR, "val-nb.json"),
#     image_dir=DATA_DIR,
#     cached=False)
# ds.caption_map
from skimage.transform import resize
from keras.applications.xception import preprocess_input
def image_transformer(image):
    return preprocess_input(resize(image, (240, 320), mode='constant', cval=0))

train_ds = ImageClassificationDataset(
    os.path.join(DATA_DIR, "training-forwards-with-bg.json"),
    image_dir=DATA_DIR,
    cached=False,preprocess_data=image_transformer,
    force_balance=True,
    translator=CocoCategoryTranslator)

val_ds = ImageClassificationDataset(
    os.path.join(DATA_DIR, "validation-forwards-with-bg.json"),
    image_dir=DATA_DIR,
    cached=False,preprocess_data=image_transformer,
    force_balance=True,
    translator=CocoCategoryTranslator)

loading annotations into memory...


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/ssd1/processed/industry-data/project-max/ml/cloud-factory-data/with-bg/multi-label-datasets/forwards/earl-originals/training-forwards-with-bg.json'

In [ ]:
dir(train_ds)

In [ ]:
image, caption = train_ds.sample()
print("sample:", image.shape, caption)
print("generated:")
for image, label in train_ds.generator(endless=False):
    print(image.shape, label)
    plt.figure()
    plt.imshow(image)
    break

image, caption = val_ds.sample()
print("sample:", image.shape, caption)
print("generated:")
for image, label in val_ds.generator(endless=False):
    print(image.shape, label)
    plt.figure()
    plt.imshow(image)
    break

image.shape

In [ ]:
catIds = train_ds.coco.getCatIds()
train_ds.print_class_stats()

# Create a Model

In [ ]:
from abyss_deep_learning.keras.models import ImageClassifier
from keras.backend import clear_session

batch_size = 10
steps_per_epoch=math.floor(len(train_ds.coco.getImgIds())/batch_size)

def create_new_model():
    model = None  # Clear any existing models
    clear_session()
    model = ImageClassifier(
        backbone='xception', input_shape=(240, 320, 3), classes=4,
        init_lr=1e-5, init_weights='imagenet',
        trainable=True)
    return model

model = create_new_model()


# Train the model

In [ ]:
from abyss_deep_learning.keras.utils import batching_gen
from abyss_deep_learning.keras.classification import onehot_gen

def pipeline(gen):
    return (onehot_gen(gen,num_classes=4))

model = create_new_model()    
print("Break-even loss is", -np.log(1 / model.classes))
model.fit_generator(batching_gen(pipeline(train_ds.generator(endless=True)),batch_size=batch_size),
    validation_data=batching_gen(pipeline(val_ds.generator(endless=True)),batch_size=batch_size), validation_steps=2,
    epochs=4, use_multiprocessing=True,steps_per_epoch=steps_per_epoch,
    verbose=True)

model.save('model-path')

In [ ]:
model = None


In [ ]:
model = ImageClassifier().load('model-path')